In [8]:
import torch
import torch.nn as nn
import torchvision
import torchvision.utils as vutils
import torch.optim as optim
import torch.utils.data as Data
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import numpy as np

下载并加载数据集

In [2]:
train_data = torchvision.datasets.MNIST(
    root = './datasets/MNIST',
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = True,
)

train_loader = Data.DataLoader(
    dataset = train_data,
    batch_size = 128,
    shuffle = True,
    num_workers = 1,
)

test_data = torchvision.datasets.MNIST(
    root = './datasets/MNIST',
    train = False,
    transform = torchvision.transforms.ToTensor(),
    download = False,
)
test_data_x = test_data.data.type(torch.FloatTensor) / 255.0
test_data_x = torch.unsqueeze(test_data_x, dim = 1)
test_data_y = test_data.targets
print(test_data_y.shape)

torch.Size([10000])


查看数据集

In [3]:
for epoch, (data, target) in enumerate(train_loader):
    if epoch > 0:
        break
    print(data.shape)
    print(target.shape)

torch.Size([128, 1, 28, 28])
torch.Size([128])


定义网络

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        #定义第一个卷积层
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels = 1,
                      out_channels = 16,
                      kernel_size = 3,
                      stride = 1,
                      padding = 1,
            ),
            nn.ReLU(),
            nn.AvgPool2d(
                kernel_size = 2,
                stride = 2,
            ),
        )
        #定义第二个卷积层
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        #定义第第一个全连接层
        self.fc = nn.Sequential(
            nn.Linear(
                in_features = 32*7*7,
                out_features = 128,
            ),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
        )
        self.out = nn.Linear(64, 10)
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        output = self.out(x)
        return output

In [ ]:
net = ConvNet()
print(net)

PyTorchViz库可视化网络结构

In [ ]:
from torchviz import make_dot
x = torch.randn(1, 1, 28, 28).requires_grad_(True)
y = net(x)
MyConvnetvis = make_dot(y, params = dict(list(net.named_parameters()) + [('x', x)]))
MyConvnetvis.format = 'png'
MyConvnetvis.directory = './nets/charpt4/net_vis'
MyConvnetvis.view()

定义优化器

In [ ]:
optimizer = optim.Adam(net.parameters(), lr = 0.00003)
criteon = nn.CrossEntropyLoss()
epochs = 10
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        output = net(data)
        loss = criteon(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step
        if batch_idx%100 == 0:
            print('Train Epoch:{}[{}/{} ({:.0f}%)]\tloss:{:.6f}'.format(epoch, batch_idx*len(data), len(train_loader.dataset),
                                                                     100.*batch_idx/len(train_loader), loss.item()))
        

使用tensorboardX进行可视化

In [ ]:
from tensorboardX import SummaryWriter
SumWriter = SummaryWriter(log_dir = './vis/charpt4/log')
optimizer = torch.optim.Adam(net.parameters(), lr = 0.0003)
criteon = nn.CrossEntropyLoss()
train_loss = 0
for epoch in range(5):
    for step, (data, target) in enumerate(train_loader):
        output = net(data)
        loss = criteon(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss = loss + train_loss
        #计算迭代次数
        niter = epoch * len(train_loader) + step +1
        if niter % 100 == 0:
            SumWriter.add_scalar('train_loss', train_loss.item()/niter, global_step = niter)
            output = net(test_data_x)
            _,pre_lab = torch.max(output, 1)
            acc = accuracy_score(test_data_y, pre_lab)
            SumWriter.add_scalar('test_acc', acc.item(), niter)
            b_x_im = vutils.make_grid(data, nrow = 12)
            SumWriter.add_image('train_image sample', b_x_im, niter)
            for name, param in net.named_parameters():
                SumWriter.add_histogram(name, param.data.numpy(), niter)

使用visdom可视化

In [5]:
from visdom import Visdom
from sklearn.datasets import load_iris
iris_x, iris_y = load_iris(return_X_y = True)
print(iris_x.shape)
print(iris_y.shape)

(150, 4)
(150,)


先启动visdom服务。
python -m visdom.server
http://localhost:8097

In [6]:
vis = Visdom()
vis.scatter(iris_x[:,0:2], Y = iris_y+1, win= 'windows1', env = 'main')
vis.scatter(iris_x[:,0:3], Y = iris_y+1, win='3D 散点图', env='main', opts = dict(markersize = 4, xlabel = '特征1', ylabel = '特征2'))


Setting up a new session...


'3D 散点图'

In [9]:
x = torch.linspace(-6, 6, 100).view((-1,1))
sigmoid = torch.nn.Sigmoid()
sigmoidy = sigmoid(x)
tanh = torch.nn.Tanh()
tanhy = tanh(x)
relu = torch.nn.ReLU()
reluy = relu(x)
ploty = torch.cat((sigmoidy,tanhy,reluy), dim=1)
plotx = torch.cat((x,x,x),dim = 1)
vis.line(Y = ploty, X = plotx, win='line plot', env = 'main', opts = dict(dash = np.array(['solid','dash','dashdot']),
                                                                        legend = ['Sigmoid', 'Tanh','ReLu']))

'line plot'

In [10]:
x = torch.linspace(-6, 6, 100).view((-1, 1))
y1 = torch.sin(x)
y2 = torch.cos(x)
plotx = torch.cat((y1, y2), dim = 1)
ploty = torch.cat((x, x), dim = 1)
vis.stem(X = plotx, Y = ploty, win = 'stem plot', env = 'main', opts = dict(legend = ['sin', 'cos'], title = '茎叶图'))


'stem plot'

In [11]:
iris_corr = torch.from_numpy(np.corrcoef(iris_x, rowvar = False))
vis.heatmap(iris_corr, win = 'heatmap', env = 'main',
            opts = dict(rownames = ['x1', 'x2', 'x3', 'x4'],
                       columnnames = ['x1', 'x2', 'x3', 'x4'],
                       title = '热力图')
           )

'heatmap'

In [13]:
for step, (data, target) in enumerate(train_loader):
    if step > 0:
        break
    vis.image(data[0,:,:,:], win = 'one image', env = 'MyimagePlot',
         opts = dict(title = '一张图像'))
    vis.images(data, win = 'batch images', env = 'MyimagePlot',nrow = 16, opts = dict(title = '一个批次的图像'))

Connection to remote host was lost.
[Errno 61] Connection refused
[Errno 61] Connection refused
[Errno 61] Connection refused
